In [1]:
from prophet import Prophet
import pandas as pd

# Sample data
data = {'ds': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05']),
        'y': [10, 12, 15, 13, 18]}
df = pd.DataFrame(data)

# Initialize and fit the model
model = Prophet()
model.fit(df)

# Create a dataframe for future predictions
future = model.make_future_dataframe(periods=7)

# Make predictions
forecast = model.predict(future)

# Print the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

10:41:43 - cmdstanpy - INFO - Chain [1] start processing
10:41:43 - cmdstanpy - INFO - Chain [1] done processing


           ds       yhat  yhat_lower  yhat_upper
7  2023-01-08  22.021515   20.428066   23.703427
8  2023-01-09  23.704362   22.068099   25.366596
9  2023-01-10  25.387209   23.853986   27.007233
10 2023-01-11  27.070056   25.330138   28.720701
11 2023-01-12  28.752903   27.194746   30.428494


In [4]:
import pickle
import os

# Create directory for models if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save the trained model
with open('models/prophet_model_test.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved successfully to models/prophet_model_test.pkl")

# Load the model back
with open('models/prophet_model_test.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print("Model loaded successfully")

# Create a dataframe for future predictions with the loaded model
future_loaded = loaded_model.make_future_dataframe(periods=7)

# Make predictions with the loaded model
forecast_loaded = loaded_model.predict(future_loaded)

# Print the forecast from loaded model
print("\nForecast from loaded model:")
print(forecast_loaded[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

# Compare results between original and loaded model
print("\nVerifying predictions are the same:")
original_tail = forecast[['ds', 'yhat']].tail()
loaded_tail = forecast_loaded[['ds', 'yhat']].tail()
comparison = original_tail.equals(loaded_tail)
print(f"Predictions match: {comparison}")

Model saved successfully to models/prophet_model_test.pkl
Model loaded successfully

Forecast from loaded model:
           ds       yhat  yhat_lower  yhat_upper
7  2023-01-08  22.021515   20.379837   23.683061
8  2023-01-09  23.704362   22.111630   25.341334
9  2023-01-10  25.387209   23.785482   27.161395
10 2023-01-11  27.070056   25.421592   28.753073
11 2023-01-12  28.752903   27.127275   30.368733

Verifying predictions are the same:
Predictions match: True


In [5]:
import os
import pandas as pd
import pickle
from prophet import Prophet
from glob import glob

def train_and_save_models():
    # Create models directory if it doesn't exist
    os.makedirs('models', exist_ok=True)
    
    # Get list of all CSV files in the data folder
    csv_files = glob('data/*.csv')
    
    if not csv_files:
        print("No CSV files found in the data folder.")
        return
    
    print(f"Found {len(csv_files)} CSV files in the data folder.")
    
    for csv_path in csv_files:
        try:
            # Get filename without extension for model naming
            filename = os.path.basename(csv_path)
            model_name = os.path.splitext(filename)[0]
            model_path = f"models/{model_name}.pkl"
            
            print(f"Processing {filename}...")
            
            # Load data from CSV file
            df = pd.read_csv(csv_path)
            
            # Ensure the dataframe has the required columns (ds, y)
            if 'ds' not in df.columns or 'y' not in df.columns:
                print(f"Error: {filename} must contain 'ds' and 'y' columns. Skipping file.")
                continue
                
            # Convert ds to datetime if it's not already
            df['ds'] = pd.to_datetime(df['ds'])
            
            # Initialize and fit the model
            model = Prophet()
            model.fit(df)
            
            # Save the trained model
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
            
            print(f"Model trained and saved successfully to {model_path}")
            
            # Optional: Verify the model by making a simple prediction
            future = model.make_future_dataframe(periods=7)
            forecast = model.predict(future)
            print(f"Sample forecast for {filename}:")
            print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(3))
            print("-" * 50)
            
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    
    print("All models training completed.")

In [6]:
train_and_save_models()

10:57:33 - cmdstanpy - INFO - Chain [1] start processing


Found 4 CSV files in the data folder.
Processing chocolates.csv...


10:57:33 - cmdstanpy - INFO - Chain [1] done processing
10:57:33 - cmdstanpy - INFO - Chain [1] start processing


Model trained and saved successfully to models/chocolates.pkl
Sample forecast for chocolates.csv:
            ds         yhat   yhat_lower   yhat_upper
161 2025-03-13  9038.564086  8681.863059  9414.219763
162 2025-03-14  9050.945224  8692.953230  9425.730352
163 2025-03-15  9063.922785  8722.895607  9436.919781
--------------------------------------------------
Processing facewash.csv...


10:57:33 - cmdstanpy - INFO - Chain [1] done processing
10:57:33 - cmdstanpy - INFO - Chain [1] start processing


Model trained and saved successfully to models/facewash.pkl
Sample forecast for facewash.csv:
            ds        yhat  yhat_lower  yhat_upper
161 2025-03-07  144.336809  141.190760  148.000448
162 2025-03-08  144.552552  141.111640  147.664000
163 2025-03-09  144.785330  141.535496  147.805293
--------------------------------------------------
Processing fairy_lights.csv...


10:57:33 - cmdstanpy - INFO - Chain [1] done processing
10:57:33 - cmdstanpy - INFO - Chain [1] start processing


Model trained and saved successfully to models/fairy_lights.pkl
Sample forecast for fairy_lights.csv:
            ds        yhat  yhat_lower  yhat_upper
147 2025-03-07  541.702780  472.271261  615.362437
148 2025-03-08  547.115305  477.824023  621.814938
149 2025-03-09  552.252603  481.270130  623.436047
--------------------------------------------------
Processing preotein.csv...


10:57:33 - cmdstanpy - INFO - Chain [1] done processing


Model trained and saved successfully to models/preotein.pkl
Sample forecast for preotein.csv:
            ds        yhat  yhat_lower  yhat_upper
160 2025-03-13  720.573888  711.086196  730.762949
161 2025-03-14  722.227614  712.403468  731.309300
162 2025-03-15  723.961602  715.308779  734.219393
--------------------------------------------------
All models training completed.
